<a id='top'></a>

------


# 2020-0120 C6 Data Science Pipeline and Critical Thinking

=====

**Instructors**: Srikant Datar, Hanspeter Pfister, Joe Blitzstein <br/>
**Author**: Rashmi Banthia

--------


In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing

In [2]:
loans = pd.read_csv('loans2013_2015q1_minimalprocessing.csv')
print(loans.shape)
loans.head()

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (49,129,130,131,134,135,136,139) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(413534, 152)


id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  28252572        NaN       8000         8000           8000.0   36 months   
1  26259270        NaN      10850        10850          10850.0   36 months   
2  16662076        NaN       6825         6825           6825.0   36 months   
3   6730044        NaN      10000        10000           9950.0   36 months   
4   4556545        NaN       5000         5000           5000.0   36 months   

  int_rate  installment grade sub_grade  ... hardship_last_payment_amount  \
0   11.67%       264.46     B        B4  ...                          NaN   
1   10.15%       350.87     B        B2  ...                          NaN   
2   18.24%       247.57     D        D5  ...                          NaN   
3   12.35%       333.82     B        B4  ...                          NaN   
4   17.77%       180.19     D        D1  ...                          NaN   

  disbursement_method debt_settlement_flag  debt_settlement_flag_date  \
0                Cash                    N                        NaN   
1                Cash                    N                        NaN   
2                Cash                    N                        NaN   
3                Cash                    N                        NaN   
4                Cash                    N                        NaN   

  settlement_status settlement_date settlement_amount settlement_percentage  \
0               NaN             NaN               NaN                   NaN   
1               NaN             NaN               NaN                   NaN   
2               NaN             NaN               NaN                   NaN   
3               NaN             NaN               NaN                   NaN   
4               NaN             NaN               NaN                   NaN   

  settlement_term good_loan  
0             NaN         0  
1             NaN         0  
2             NaN         0  
3             NaN         0  
4             NaN         0  

[5 rows x 152 columns]

In [3]:
loans.loan_status.value_counts(normalize=True)

Fully Paid     0.820298
Charged Off    0.179615
Default        0.000087
Name: loan_status, dtype: float64

In [4]:
print(list(loans.columns))

['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq',

In [5]:
#drop leakage columns
loans.drop(['out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 
         'total_rec_prncp','total_rec_int','total_rec_late_fee', 'recoveries','collection_recovery_fee',
         'last_pymnt_amnt',
        'debt_settlement_flag',
         'disbursement_method', 'funded_amnt', 'funded_amnt_inv', 'installment', 'issue_d', 'last_pymnt_d' , 
         'pymnt_plan' , 'term' ,  
        'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low'
        ],axis=1,inplace=True) 

print("\n","Loans.shape: ", loans.shape)


 Loans.shape:  (413534, 130)


In [6]:
loans.reset_index(inplace=True,drop=True)

#drop

lack_of_data_idx = [x for x in loans.count() < loans.shape[0]*0.25]
loans.drop(loans.columns[lack_of_data_idx], 1, inplace=True)

loans.drop('id',axis=1,inplace=True)

print(loans.shape,loans.columns)

loans.reset_index(inplace=True,drop=True)

#drop

lack_of_data_idx = [x for x in loans.count() < loans.shape[0]*0.25]
loans.drop(loans.columns[lack_of_data_idx], 1, inplace=True)

print("\nAfter dropping columns with <25% data:",loans.shape,loans.columns)

(413534, 75) Index(['loan_amnt', 'int_rate', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'loan_status', 'url', 'purpose', 'title', 'zip_code', 'addr_state',
       'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low',
       'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_deli

In [7]:
loans.verification_status.value_counts()

Source Verified    144197
Verified           141802
Not Verified       127535
Name: verification_status, dtype: int64

In [8]:
loans.purpose.value_counts()

debt_consolidation    249729
credit_card            98094
home_improvement       22394
other                  18585
major_purchase          6914
small_business          4048
medical                 3692
car                     3312
moving                  2286
vacation                2067
house                   1612
wedding                  592
renewable_energy         209
Name: purpose, dtype: int64

In [9]:
#Convert purpose to dummies
purpose_cols = pd.get_dummies(loans.purpose).columns
print(pd.unique(loans.purpose))
loans = pd.concat([loans,pd.get_dummies(loans.purpose)],axis=1)
loans.drop('purpose',axis=1,inplace=True)

#Convert emp_length to numeric
loans['emp_length'] = loans.emp_length.map({'< 1 year':0,'1 year':1,'2 years':2 , 
                            '3 years':3,'4 years':4, '5 years':5,
                           '6 years':6, '7 years':7, '8 years':8,
                           '9 years':9, '10+ years':10})

#DROP EMP_TITLE
loans.drop(['emp_title'], axis=1, inplace = True)

#Convert Verification Status
verification_status_cols = pd.get_dummies(loans.verification_status).columns
loans = pd.concat([loans,pd.get_dummies(loans.verification_status)],axis=1)
loans.drop('verification_status',axis=1,inplace=True)
print(verification_status_cols)

#Convert categorical data
categorical = list(loans.select_dtypes(include='object').columns)
#convert grade and subgrade later
categorical = ['home_ownership', 'url', 'title', 'zip_code', 'addr_state', 
               'earliest_cr_line', 'revol_util', 'initial_list_status',
               #'last_credit_pull_d',
               'application_type', 'hardship_flag']

print(categorical)

# Encoder:
lbl = preprocessing.LabelEncoder()
for col in categorical:
    loans[col] = lbl.fit_transform(loans[col].astype(str))
    
loans = loans.fillna(-999)
print(loans.shape)

['debt_consolidation' 'credit_card' 'small_business' 'major_purchase'
 'house' 'other' 'home_improvement' 'medical' 'car' 'vacation' 'moving'
 'wedding' 'renewable_energy']
Index(['Not Verified', 'Source Verified', 'Verified'], dtype='object')
['home_ownership', 'url', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'application_type', 'hardship_flag']
(413534, 88)


In [10]:
loans.dtypes

loan_amnt                        int64
int_rate                        object
grade                           object
sub_grade                       object
emp_length                     float64
home_ownership                   int64
annual_inc                     float64
loan_status                     object
url                              int64
title                            int64
zip_code                         int64
addr_state                       int64
dti                            float64
delinq_2yrs                      int64
earliest_cr_line                 int64
fico_range_low                   int64
fico_range_high                  int64
inq_last_6mths                   int64
mths_since_last_delinq         float64
open_acc                         int64
pub_rec                          int64
revol_bal                        int64
revol_util                       int64
total_acc                        int64
initial_list_status              int64
collections_12_mths_ex_me

In [11]:
loans['int_rate'] = loans['int_rate'].str.replace('%','')
loans['int_rate'] = loans['int_rate'].astype(np.float32)


In [12]:
categorical = ['grade', 'sub_grade']

lbl = preprocessing.LabelEncoder()
for col in categorical:
    loans[col] = lbl.fit_transform(loans[col].astype(str))

In [13]:
loans.shape

(413534, 88)

In [14]:
loans.columns

Index(['loan_amnt', 'int_rate', 'grade', 'sub_grade', 'emp_length',
       'home_ownership', 'annual_inc', 'loan_status', 'url', 'title',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'fico_range_low', 'fico_range_high', 'inq_last_6mths',
       'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'initial_list_status',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths',
       'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths',
       'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op',
       'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
       'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_

In [15]:
#Dropping because we have good_loan as target variable
loans.drop('loan_status', axis=1,inplace=True)

In [16]:
loans.shape

(413534, 87)

In [17]:
loans.to_csv('loans_dsp_2019.csv', index=False)
